# 1.Introduction

This projects is coming as a subproject for Kaggle's competetion: `Two Sigma: Using News to Predict Stock Movements`.
Basically its task is to predict if a given information/news brings positive or negative connotations.

This model may be/will be used for predicting how a given information coming from News data may affect a company's stock prices. 

# 2.Modules

## 2.1 NLTK modules for language processing 

In [132]:
import numpy as np
import pandas as pd
import random
import nltk
from sklearn.cross_validation import train_test_split
import re
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import state_union, wordnet, stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer #Stemming words e.g Writing -> Write
"""Models for training and testing the results"""
from sklearn.naive_bayes import GaussianNB

## 2.2 Downloading NLTK submodules

In [121]:
nltk.download('stopwords')
nltk.download('movie_reviews')
stopwords= stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\JA\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [127]:
df = pd.read_csv('./data/data.tsv', delimiter = '\t')
n_sentences = len(df)
feature = 'Review'
label = 'Liked'

In [123]:
corpora = []
ps = PorterStemmer()
for review_it in range(0, n_sentences):
    review = re.sub('[^a-zA-Z]', ' ', dataframe[feature][review_it])
    review = review.lower()
    review = review.split()
    stemmed = []
    for word in review:
        if not word in stopwords:
            stemmed.append(ps.stem(word))
    review = stemmed
    review = ' '.join(review)
    corpora.append(review)    

In [147]:
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(corpora).toarray()
Y = df[label].values

In [148]:
X_train, X_test, Y_train, Y_test  = train_test_split(X, Y, test_size = 0.15, random_state = 42)

In [149]:
classifier = GaussianNB()
classifier.fit(X_train, Y_train)

GaussianNB(priors=None)

In [150]:
y_pred = classifier.predict(X_test)

In [151]:
y_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int64)

In [152]:
Y_test

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0], dtype=int64)

In [153]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)

In [154]:
cm

array([[33, 38],
       [13, 66]], dtype=int64)